# OpenRouteService Optimization Routing examples

This notebook is just a small reference on how to use `openrouteservice` package to request:
- [directions](https://openrouteservice.org/dev)
- [Route Optimization](https://openrouteservice.org/dev/#/api-docs/optimization/post)

In [140]:
import openrouteservice as ors
import folium
import json
from pprint import pprint
#import requests

client = ors.Client(base_url='http://ors:8080/ors', key='')

povoLoc = [46.064424,11.157259]

center = povoLoc

#remote https://gist.github.com/stefanocudini/aac0062312b743643a16c30c3970eca9
#contains two features MultiPoint
features = json.load(open('./povo.geojson'))['features']

In [165]:
points = {
    'A': features[0]['geometry']['coordinates'],
    'B': features[1]['geometry']['coordinates']
}

pprint(points)

{'A': [[11.158568859100342, 46.06463328438495],
       [11.15638017654419, 46.06304021008937],
       [11.151916980743408, 46.06212454377845]],
 'B': [[11.146938800811768, 46.06608487747521],
       [11.15655183792114, 46.067261012088416],
       [11.15309715270996, 46.06770763892092],
       [11.153333187103271, 46.071206090751375]]}


## Directions

In [158]:
M = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=16)

routeA = client.directions(
    coordinates = points['A'],
    profile = 'driving-car',
    format = 'geojson',
    #options={"avoid_features": ["steps"]},
    validate=False
)

routeB = client.directions(
    coordinates = points['B'],
    profile = 'driving-car',
    format = 'geojson',
    #options={"avoid_features": ["steps"]},
    validate=False
)

polyA = [list(reversed(coord)) for coord in routeA['features'][0]['geometry']['coordinates']]
polyB = [list(reversed(coord)) for coord in routeB['features'][0]['geometry']['coordinates']]

folium.PolyLine(polyA, color='red').add_to(M)
folium.PolyLine(polyB, color='blue').add_to(M)
#pprint(points['A'])

for c in points['A']:
    m = list(reversed(c))
    folium.Marker(m, icon=folium.Icon(color='red')).add_to(M)

for c in points['B']:
    m = list(reversed(c))
    folium.Marker(m).add_to(M)
M

## Simple waypoint optimization

With `openrouteservice-py` you can ask for waypoint optimization when requesting a normal `direction`. This assumes that your first coordinate is the start location and the last coordinate is the end location, i.e. only the `via` endpoints are optimized. To make it a round trip, make the first and last location the same.

In [170]:
#M = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=16)

routeO = client.directions(
    coordinates = points['B'],
    profile = 'driving-car',
    format = 'geojson',
    validate = False,
    optimize_waypoints = True
    #TODO not works
)
polyO = [list(reversed(coord)) for coord in routeO['features'][0]['geometry']['coordinates']]

folium.PolyLine(polyO, color='green').add_to(M)

M

AttributeError: 'module' object has no attribute 'JSONDecodeError'

### Optimize job scheduling for multiple vehicles

The next example makes more use of the power of the ORS optimization endpoint. Two vehicles are assigned to 6 jobs, where the capacity constraints are such that each vehicle can only carry out 3 jobs on its route.

In [24]:
m = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=16)

vehicle_locations = [[13.390446, 52.506087], [13.384116, 52.533558]]
job_locations = [[13.428726, 52.519355],
                 [13.41774, 52.498929],
                 [13.374825, 52.496369],
                 [13.378859, 52.509796],
                 [13.400488, 52.509691],
                 [13.358517, 52.524264]]

# Assign vehicles to do the jobs
vehicles = []
for idx, coords in enumerate(vehicle_locations):
    vehicles.append(ors.optimization.Vehicle(
        id=idx,
        profile='driving-car',
        start=coords,
        end=coords,
        capacity=[3]  # Limit capacity so only 3 jobs can be taken by each vehicle
    ))
    folium.Marker(location=list(reversed(coords)), icon=folium.Icon(icon='truck', prefix='fa')).add_to(m)

# Define jobs to be carried out
jobs=[]
for idx, coords in enumerate(job_locations):
    jobs.append(ors.optimization.Job(
        id=idx,
        location=coords,
        amount=[1]  # Occupies capacity in vehicle
    ))
    folium.Marker(location=list(reversed(coords)), icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(m)

optimized = client.optimization(
    jobs=jobs,
    vehicles=vehicles,
    geometry=True,  ## will output the geometry,
)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][0]['geometry'])['coordinates']],
    color='red'
).add_to(m)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(optimized['routes'][1]['geometry'])['coordinates']],
    color='orange'
).add_to(m)

m